# Directions

Hello! This python script will help you clean address data for use in analysis.

Currently, the script is designed to run on a pre-cleaned set of data on Eviction Filings supplied by the City of Grand Rapids. The pre-cleaned sheet should have blank rows, c/o address lines, and P.O. Boxes removed and should be filtered to include only addreses with a ZIP Code in Kent County. Additional work can be done to make sure each combination of City, State, and ZIP Code data are free of typographical errors--i did this by taking the unique values from the entire spreadsheet for City, City and State, and City and ZIP Code and made changes by hand. This kind of finnicky work saved immeasureable time on the backend.

From the outset you should set aside and afternoon or even a full day (depending on your comfort with excel or another data reviewing tool) to preparing the dataset, then running it through this script, and finally reviewing missing addresses in the Final Export Document.

Make sure Python 3 (any version) is installed on your device. For instructions on downloading Python, see here: https://www.dummies.com/article/technology/programming-web-design/python/how-to-install-python-on-your-computer-139548/ .

To generate and set a Google Maps API key (for FREE!) follow the instructions in this link: https://yoast.com/help/generate-set-google-maps-api-key/ .

The geopy code used in this notebook is adapted from code written by Aaron Zhu in "Clean Messy Address Data Effortlessly Using Geopy and Python" for Towards Data Science. Read the original article here: https://medium.com/towards-data-science/transform-messy-address-into-clean-data-effortlessly-using-geopy-and-python-d3f726461225 .

# Upload a Dataset

This program accepts files in all standard excel formats (.xls, .xlsx, .csv, and others).

In the first section, you will need to enter information to populate our variables. Be careful not to delete anything on the left of the equal signs ("=")—those are variable names and we will need them exactly as written for the code to work.

In [ ]:
# Enter *SOURCE* FOLDER address from file directory in quotation marks--
## the folder that holds your file
## make sure all slashes are BACKslashes ("\") and 
## that there are 2 backslashes after the drive (e.g., "C:\\")
folder_source = "C:\\YOUR_DIRECTORY\YOUR_FOLDER"

# Enter the FILE NAME *with EXTENSION* in quotation marks (e.g., "evicitions_file.xlsx")
## this will not work without the extension at the end, so verify it is added
file = "YOUR_FILE.xlsx"

# If you have more than 1 sheet in your target file, type the name 
## of the sheet you would like to use from the workbook in quotation marks
## if there is no sheet, just enter blank quotation marks (e.g., "")
sheet = "SHEET_NAME"

# Enter the COLUMN NAMES that hold Street Address, City, State, and ZIP Code from your dataset
## so we can match the correct columns later
street = "STREET_ADDRESS"
city = "CITY"
state = "STATE"
zip_code = "ZIP_CODE"

# Enter *SAVE* FOLDER address from file directory in quotation marks--
## the folder into which you want a the cleaned data saved
## make sure all slashes are BACKslashes ("\") and 
## that there are 2 backslashes after the drive (e.g., "C:\\")
## set equal to folder_source by default
folder_save = folder_source

# Enter your key for accessing the Google Maps API
google_cloud_key = "YOUR_KEY_HERE"


In the next section, we will !pip install the pandas library from PyPI. If you already have this installed, skip to the next section.

In [ ]:
!pip install pandas

If you successfully installed pandas or you have it previously installed, move on to the next code section.

In this section we will load pandas to our environment and create and open our dataframe.

In [ ]:
import pandas as pd

workbook = folder_source + "\\" + file
if workbook[-4:-1] == ".cs":
    addressesDF = pd.read_csv(workbook)
else:
    addressesDF = pd.read_excel(workbook,sheet_name = sheet,engine = "openpyxl")

df = addressesDF 
print(df)

##### Be sure to check that the result from the previous code has 7 columns and the same number of rows as your data source.

## Formatting as List and Cleaning

Next we will take a unique list of addresses from the dataset to save some compute when we pass the information to the Google Maps API. Every 1000 addresses processed costs $5 and it adds up fast. We want to stay well below our free monthly limit.

In [ ]:
df['ADDRESS'] = df[street] + ", " + df[city] + ", " + df[state] + ' ' + df[zip_code].astype(str)
df_unique = pd.DataFrame(pd.unique(df['ADDRESS']),columns = ['Old_Addresses']).dropna()
print(df_unique)

Check to see that your results look right. If not, you may need to adjust your variables at the top of the notebook or adjust something in the source file.

## Installing GeoPy and Pinging the API

In the next section, we will !pip install the geopy and re library from PyPI. If you already have these installed, skip to the next section.

In [ ]:
!pip install geopy
!pip install re

If you successfully installed pandas or you have it previously installed, move on to the next code section.

In this section we will load geopy, make our connection to the Google Maps API, pass our data to the API, and save information from each call.

In [ ]:
import geopy
import re

from geopy.geocoders import GoogleV3
geolocator = GoogleV3(api_key=google_cloud_key)

def extract_clean_address(address):
    try:
        location = geolocator.geocode(address)
        return [location.address, location.latitude, location.longitude]
    except:
        return ''

df_unique['add_lat_long'] = df_unique.apply(lambda x: extract_clean_address(x['Old_Addresses']), axis = 1)    
df_unique['Address_F'] = df_unique.apply(lambda x: x['add_lat_long'][0] if x['add_lat_long'] != '' else '', axis = 1)
df_unique['Latitude_F'] = df_unique.apply(lambda x: x['add_lat_long'][1] if x['add_lat_long'] != '' else '', axis = 1)
df_unique['Longitude_F'] = df_unique.apply(lambda x: x['add_lat_long'][2] if x['add_lat_long'] != '' else '', axis = 1)
df_unique.drop(columns = ['add_lat_long'],inplace = True)

print(df_unique)

csvFile = folder_save + "\\" + re.sub("\..*","",file) + "_uniqAddressesCl.csv"
pd.DataFrame.to_csv(df_unique, path_or_buf = csvFile, encoding = 'utf-8', index = False)

The previous code module may take a few minutes. Once finished, I recommend opening the file containing the unique old addresses with results from the api for each. The file should be called "\[file name\]\_uniqAddressesCl.csv" and located in the file_save folder. Especially check for blanks in Address_F, Latitude_F, and Longitude_F. Blanks signify that the address in questions was not accepted by the algorithm. If you have time and some free compute left from Google Cloud Platform, you may want to try the following. Otherwise skip to the next header and continue from there.

> As an aside—and only if your blanks count greater than 1% of the dataset—one could subset these addresses and paste them to a new file in order to review them. If you can use your human skills to clean them at all and then replace the original in the source file, you can rerun this notebook from the start to see if your edits lead to better results. But, you only want to this once, so be definitive in your changes. Re-running more than once may cause you to use up your free compute from Google Cloud Platform and force you to pay out of pocket.



## Merging New Data and Old Data; Final Check

Next, we will left join the columns Address_F, Latitude_F, and Longitude_F to all rows in the original dataset.

In [ ]:
df_final = df.merge(
                df_unique,
                how = "left",
                left_on = 'ADDRESS',
                right_on = 'Old_Addresses'
            ).drop(columns = ['Old_Addresses'])


print(df_final)

Check the printed results for errors. This is only a cursory check of 10 rows, but problems should be clearly evident: blank cells, incorrect matches, etc.

If satisfied, proceed to the next code block to save the file as a .xlsx workbook.

In [ ]:
finalFile = folder_save + "\\" + re.sub("\..*","",file) + "_clFinal.csv"
pd.DataFrame.to_csv(df_final, path_or_buf = finalFile, encoding = 'utf-8', index = False)

## Congrats!

That is it! All you have to now is open your file and check your data. If you have issues, questions, concerns, or reccomendations, email me at hileevanw@gmail.com